In [1]:
import requests
import json
import pandas as pd
import datetime
import os

In [2]:
# NOTE: Make sure to include a & symbol at the end of the url for it to retrive the correct results properly.
api_link_specific = """
https://www.tes.com/api/jobs/browser/search-v3?siteCountry=gb&&locations=United%20Kingdom%3AEngland%3AYorkshire%20and%20the%20Humber%3ALeeds&keywords=&workplaces=Secondary&positions=Teaching%20and%20Lecturing&
"""

In [3]:
request  =requests.get(api_link_specific, headers={"referer": "https://www.tes.com/jobs/browse/"})

In [4]:
if request.status_code == 200:
    raw_data = json.loads(request.text)
else:
    raise requests.HTTPError(f"Could not find webpage (status code: {request.status_code}")

In [5]:
raw_jobs_data = raw_data["searchResult"]["jobs"]["items"]

In [15]:
raw_data.keys()
#raw_data["searchResult"]
#raw_data["searchResult"]["jobs"]["metadata"]["pagination"]["totalpages"]

dict_keys(['searchResult', 'savedJobs', 'searchDetails', 'selectedFacets', 'savedSearches', 'baseQS'])

In [7]:
print(raw_jobs_data[0].keys())

dict_keys(['lastModified', 'advert', 'title', 'package', 'employerOnlineId', 'attachments', 'hasAttachments', 'deleted', 'shortDescription', 'map', 'employerName', 'jobId', 'jobStats', 'modulesV2', 'cardStyle', 'cdnBase', 'flag', 'jobPageCanonicalUrl', 'displayTitle', 'displayJobStartDate', 'displaySalary', 'salary', 'displayLocation', 'displayContractTerms', 'application', 'quickApplication', 'employer'])


In [8]:
first_item = raw_jobs_data[2]

print(first_item["title"])
print(first_item["employerName"])
print(first_item["displayLocation"])
print(first_item["displaySalary"]) # This requires more parsing (if exists, get min, max)
print(first_item["jobPageCanonicalUrl"])
print(first_item["displayJobStartDate"])
print(first_item["advert"]["displayEndDateShort"]) # Perhaps convert to datetime format before adding to df
print(first_item["displayContractTerms"])

"""
for key in list(first_item.keys()):
    print(f"{key}: {first_item[key]}")
"""

Inclusion Learning Support Assistant
Mount St Mary's Catholic High School
Leeds
Scale A1/B1 - £18,328 to £19,698 per annum pro rata (pro rata salary £11,903 to £12,792) starting salary dependent upon experience
https://www.tes.com/jobs/vacancy/inclusion-learning-support-assistant-leeds-1712637
As soon as possible
2022-11-02
Temporary


'\nfor key in list(first_item.keys()):\n    print(f"{key}: {first_item[key]}")\n'

In [9]:


for job in raw_jobs_data:
    print(job["advert"]["displayEndDateShort"])

2022-11-09
2022-11-03
2022-11-02
2022-11-02
2022-11-07
2022-11-07
2022-11-07
2022-11-01
2022-10-30
2022-11-07
2022-10-30
2022-10-31
2022-11-07
2022-11-30
2022-11-11
2022-11-18
2022-11-18
2022-11-14
2022-11-04
2022-11-01


In [10]:
column_names = ["job_id", "job_title", "tenure", "start_date", "application_deadline", "employer_name", "location", "salary", "url"]

In [11]:
df = pd.DataFrame(columns=column_names)

In [12]:
for job in raw_jobs_data:
    row = [job["jobId"], job["title"], job["displayContractTerms"], job["displayJobStartDate"], 
    job["advert"]["displayEndDateShort"], job["employerName"], job["displayLocation"], job["displaySalary"], job["jobPageCanonicalUrl"]]

    # Inserts row to the end of the dataframe
    df.loc[len(df)] = row

In [13]:
time = datetime.datetime.now()
timestamp = time.strftime("%Y%m%d-%H%M%S")

In [14]:
df.to_csv(f"{os.path.dirname(os.getcwd())}\\data/job_data_{timestamp}.csv", index=False)